# First steps for Google Places API

Places API supports two API versions: Places API are the existing APIs and Places API (New) are the next generation version of the APIs
https://developers.google.com/maps/documentation/places/web-service/choose-api

In the following several options to use both Places APIs are explored:

## Preparations

In [15]:
#!pip install -r requirements.txt

In [16]:
import requests
from urllib.parse import urlencode
import googlemaps
import os
from dotenv import load_dotenv
import time
import pandas as pd

In [17]:
# import the api_key from the api_key.py file
from api_key import api_key

## 1. nearbysearch: only location

In [18]:
def get_place_info(address):
# Base URL
  base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# Parameters in a dictionary
  params = {
   "input": address,
   "inputtype": "textquery",
   "fields": "name,geometry/location,place_id",
   "key": api_key,
  }
# Send request and capture response
  response = requests.get(base_url, params=params)
# Check if the request was successful
  if response.status_code == 200:
    return response.json()
  else:
    return None

In [19]:
info = get_place_info("tübingen")	
print(info)

longitude =  info["candidates"][0].get("geometry", {}).get("location", {}).get("lng")
latitude = info["candidates"][0].get("geometry", {}).get("location", {}).get("lat")

{'candidates': [{'geometry': {'location': {'lat': 48.5216364, 'lng': 9.0576448}}, 'name': 'Tübingen', 'place_id': 'ChIJgdDN7dT6mUcRjacz_s6uCKw'}], 'status': 'OK'}


In [20]:
# all fields that are possible and could be interesting for us
field_nearby_places = [
    # Fields that trigger the Nearby Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.attributions", "places.businessStatus", "places.containingPlaces", 
    "places.displayName", "places.formattedAddress", "places.googleMapsLinks", 
    "places.id", "places.location", "places.name", "places.plusCode", 
    "places.primaryType", "places.primaryTypeDisplayName", "places.pureServiceAreaBusiness", 
    "places.shortFormattedAddress", "places.subDestinations", "places.types", 
    "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Nearby Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Nearby Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

In [21]:
def get_nearby_places(latitude, longitude, radius=500, max_results=10):
    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchNearby"
    # JSON request body
    body = {"includedTypes": ["restaurant"],  # Restrict to restaurants only
            "maxResultCount": max_results,  # Max number of results to return
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": latitude,
                               "longitude": longitude},
                    "radius": radius  # Radius for the search
                }
            },
            "languageCode": "en",  # Language code for the results
        }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.name,places.id,places.displayName" # to save quota
        #"X-Goog-FieldMask": ",".join(field_nearby_places) # to get all intersting fields
    }
    # Send the POST request to the API
    response = requests.post(base_url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [22]:
result = get_nearby_places(latitude, longitude, max_results=15, radius=100)
print(result)

{'places': [{'name': 'places/ChIJn5ANpsv7mUcRGS2GWvH2-qs', 'id': 'ChIJn5ANpsv7mUcRGS2GWvH2-qs', 'displayName': {'text': '1821 Tübingen', 'languageCode': 'de'}}, {'name': 'places/ChIJZ9kADtP6mUcRF3IM1orJ8aI', 'id': 'ChIJZ9kADtP6mUcRF3IM1orJ8aI', 'displayName': {'text': 'Die Wurstküche', 'languageCode': 'de'}}, {'name': 'places/ChIJ5bNxBdP6mUcRoihNgmDzZl4', 'id': 'ChIJ5bNxBdP6mUcRoihNgmDzZl4', 'displayName': {'text': 'Esszimmer', 'languageCode': 'de'}}, {'name': 'places/ChIJraX0HdP6mUcRb9JGr9jEChQ', 'id': 'ChIJraX0HdP6mUcRb9JGr9jEChQ', 'displayName': {'text': 'Kado-Ya', 'languageCode': 'en'}}, {'name': 'places/ChIJo_FWZ9P6mUcRs-zk6oSThbI', 'id': 'ChIJo_FWZ9P6mUcRs-zk6oSThbI', 'displayName': {'text': 'Eiscafé San Marco Tübingen', 'languageCode': 'de'}}, {'name': 'places/ChIJQ4bnEdP6mUcR3w2eH4yFf1c', 'id': 'ChIJQ4bnEdP6mUcR3w2eH4yFf1c', 'displayName': {'text': 'Salam Imbiss Tübingen', 'languageCode': 'de'}}, {'name': 'places/ChIJq6q2ANP6mUcRtcP54mcEUFY', 'id': 'ChIJq6q2ANP6mUcRtcP54mcEUFY'

## 2. textsearch: keywords + opt. location
https://developers.google.com/maps/documentation/places/web-service/text-search#text-search-requests

filtering options:
- includePureServiceAreaBusinesses: filter for physical business location true or false
- locationRestriction or locationBias --> if omited API uses IP biasing by default; but locationBias parameter can be overridden if a location is in the textQuery 
    - rectangular Viewport or as a circle
- minRating
- openNow
- priceLevels
- rankPreference
- regionCode

In [23]:
# all fields that are possible and interesting for us
field_textsearch = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.attributions", "places.id", "places.name", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", 
    "places.plusCode", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.shortFormattedAddress", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.routingSummaries", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

In [24]:
def get_places_textsearch(query, radius=None, max_results=10, page_token=None):
    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    # JSON request body
    body = {
        "includedType": "restaurant",  # Restrict to restaurants only
        "strictTypeFiltering": True,  # Only return results of the specified type
        "textQuery": query,
        "pageSize": max_results,  # Specify max results per page
        "pageToken": page_token,  # Token for the next page of results
        "languageCode": "en",  # Language code for the results
    }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.name,places.id,places.displayName,nextPageToken" # to save quota
        #"X-Goog-FieldMask": ",".join(field_textsearch) # to get all intersting fields
    }
    # Send the POST request to the API
    response = requests.post(base_url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [25]:
res = get_places_textsearch("spicy vegetarian food in Tübingen", max_results=5)
res

{'places': [{'name': 'places/ChIJ8WhdiDT7mUcRJ77WT23aNh8',
   'id': 'ChIJ8WhdiDT7mUcRJ77WT23aNh8',
   'displayName': {'text': 'Köstlich Vegan', 'languageCode': 'de'}},
  {'name': 'places/ChIJu9cZ7YvlmUcRRPS7OxAUI6k',
   'id': 'ChIJu9cZ7YvlmUcRRPS7OxAUI6k',
   'displayName': {'text': 'Spicetripping Foodtruck', 'languageCode': 'de'}},
  {'name': 'places/ChIJcQumZc_6mUcRvT91eF5c3rM',
   'id': 'ChIJcQumZc_6mUcRvT91eF5c3rM',
   'displayName': {'text': 'Bongoroots', 'languageCode': 'en'}},
  {'name': 'places/ChIJm7waYdT6mUcRxPFyE982gE0',
   'id': 'ChIJm7waYdT6mUcRxPFyE982gE0',
   'displayName': {'text': 'Veggi', 'languageCode': 'de'}},
  {'name': 'places/ChIJYR6MeNP6mUcRX0PfqXYXqks',
   'id': 'ChIJYR6MeNP6mUcRX0PfqXYXqks',
   'displayName': {'text': 'Kichererbse', 'languageCode': 'de'}}],
 'nextPageToken': 'AdDdOWo_9lmIp8p2HjvB3HvfwiVH6ORVGbDb2CKl4ygwqUhUBQXQ9XcTKuaafOHmnF5cIshJawPa5THYHktd1YMY1tRhujz1u5DR3bTtxUo96rv3bBVbekO6HIp-vZdnorafKAXxxVv13Qkwquap7qIOfYFchAum_mwdu7ipxE9vvjz0n-b8m2RYHaN

In [26]:
token = res['nextPageToken']
res2 = get_places_textsearch("spicy vegetarian food in Tübingen", max_results=5, page_token=token)
res2

{'places': [{'name': 'places/ChIJRzFgy-PlmUcRzLsMfKylwKU',
   'id': 'ChIJRzFgy-PlmUcRzLsMfKylwKU',
   'displayName': {'text': 'Veggie Box', 'languageCode': 'de'}},
  {'name': 'places/ChIJ5bNxBdP6mUcRoihNgmDzZl4',
   'id': 'ChIJ5bNxBdP6mUcRoihNgmDzZl4',
   'displayName': {'text': 'Esszimmer', 'languageCode': 'de'}},
  {'name': 'places/ChIJR70z35n7mUcRCflCXpWQqGc',
   'id': 'ChIJR70z35n7mUcRCflCXpWQqGc',
   'displayName': {'text': 'Restaurant La Médina', 'languageCode': 'de'}},
  {'name': 'places/ChIJoyQjbSr7mUcR60G1VPy0I3g',
   'id': 'ChIJoyQjbSr7mUcR60G1VPy0I3g',
   'displayName': {'text': 'Maharaja Imbiss', 'languageCode': 'de'}},
  {'name': 'places/ChIJQfCVYy7lmUcRLciIbQqplJo',
   'id': 'ChIJQfCVYy7lmUcRLciIbQqplJo',
   'displayName': {'text': 'Restaurant Le Romarin', 'languageCode': 'en'}}],
 'nextPageToken': 'AdDdOWr4hP3Z51tRIAiUFBXcqY1xlK9HPgcFxn4ccLXWcAT4n8kHKAv8gVgEP1VHo8m--oIJFrDrlnJZmw10Gh0rD8m5zQpiulESwqc7k7KpUVk8VayHBzNY-r9DiCLQ1vI4n35KlTc1xcOwu_Qwe_e2WqO-zdurDBtCeJ5hT6tgJoE

## 3. place details
https://developers.google.com/maps/documentation/places/web-service/place-details 

remeber:  Generate Google Maps link
f"https://www.google.com/maps/place/?q=place_id:{place_id}"
or under 'googleMapsLinks'\'reviewsUri'

In [27]:
field_place_details = ["id", "displayName", "name", "addressComponents", "adrFormatAddress", "formattedAddress","location", 
                    "types", "accessibilityOptions", "businessStatus", "containingPlaces", "displayName", "googleMapsLinks", 
                    "primaryType", "primaryTypeDisplayName", "pureServiceAreaBusiness", "subDestinations", "currentOpeningHours", 
                    "nationalPhoneNumber", "priceLevel", "priceRange", "rating", "regularOpeningHours", "regularSecondaryOpeningHours", "userRatingCount", 
                    "websiteUri",
                    "allowsDogs", "curbsidePickup", "delivery", "dineIn", "editorialSummary", "evChargeOptions", "fuelOptions", 
                    "goodForChildren", "goodForGroups", "goodForWatchingSports", "liveMusic", "menuForChildren", "parkingOptions", 
                    "paymentOptions", "outdoorSeating", "reservable", "restroom", "reviews", "servesBeer", "servesBreakfast", 
                    "servesBrunch", "servesCocktails", "servesCoffee", "servesDessert", "servesDinner", "servesLunch", 
                    "servesVegetarianFood", "servesWine", "takeout"]

def get_place_details(place_id):
    # Base URL
    base_url = f"https://places.googleapis.com/v1/places/{place_id}"
    # Query parameters
    params = {
        "languageCode": "en"  # This is a query parameter, not a header
    }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": ",".join(field_place_details) 
    }
    # Send request and capture response
    response = requests.get(base_url, headers=headers, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [28]:
details = get_place_details("ChIJGTtrv9P6mUcRtLRexsK4n1E")
details

{'name': 'places/ChIJGTtrv9P6mUcRtLRexsK4n1E',
 'id': 'ChIJGTtrv9P6mUcRtLRexsK4n1E',
 'types': ['italian_restaurant',
  'restaurant',
  'point_of_interest',
  'food',
  'establishment'],
 'nationalPhoneNumber': '07071 25157',
 'formattedAddress': 'Bursagasse, 72070 Tübingen, Germany',
 'addressComponents': [{'longText': 'Bursagasse',
   'shortText': 'Bursagasse',
   'types': ['route'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'Tübingen',
   'types': ['locality', 'political'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'Tübingen',
   'types': ['administrative_area_level_3', 'political'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'TÜ',
   'types': ['administrative_area_level_2', 'political'],
   'languageCode': 'de'},
  {'longText': 'Baden-Württemberg',
   'shortText': 'BW',
   'types': ['administrative_area_level_1', 'political'],
   'languageCode': 'de'},
  {'longText': 'Germany',
   'shortText': 'DE',
   

# Get the data for one city

## 1. Get city boundaries and perform a grid

In [29]:
# Function to get the city boundaries
def get_city_boundaries(city_name):
    # Initialize the Google Maps client
    gmaps = googlemaps.Client(key=api_key) # googlemaps package
    
    # Get the city boundaries
    geocode_result = gmaps.geocode(city_name)

    # save the boundaries
    low_lat = geocode_result[0]['geometry']['bounds']['southwest']['lat']
    low_long = geocode_result[0]['geometry']['bounds']['southwest']['lng']
    high_lat = geocode_result[0]['geometry']['bounds']['northeast']['lat']
    high_long = geocode_result[0]['geometry']['bounds']['northeast']['lng']

    
    return low_lat, low_long, high_lat, high_long

In [30]:
tueb_bound = get_city_boundaries('Tübingen')
print(tueb_bound)

(48.4505798, 8.9644852, 48.5937085, 9.131081600000002)


In [31]:
# Function to divide the city into a grid
def divide_area_in_grid(boundary, step_size = 0.01):
    low_lat, low_long, high_lat, high_long = boundary
    grid = []
    lat = low_lat
    while lat < high_lat:
        long = low_long
        while long < high_long:
            cell = (lat, long, min(lat + step_size, high_lat), min(long + step_size, high_long))
            grid.append(cell)
            long += step_size
        lat += step_size
    return grid

In [32]:
tueb_grid = divide_area_in_grid(tueb_bound)
print(len(tueb_grid))
print(tueb_grid)

255
[(48.4505798, 8.9644852, 48.4605798, 8.9744852), (48.4505798, 8.9744852, 48.4605798, 8.9844852), (48.4505798, 8.9844852, 48.4605798, 8.9944852), (48.4505798, 8.9944852, 48.4605798, 9.0044852), (48.4505798, 9.0044852, 48.4605798, 9.0144852), (48.4505798, 9.0144852, 48.4605798, 9.024485199999999), (48.4505798, 9.024485199999999, 48.4605798, 9.034485199999999), (48.4505798, 9.034485199999999, 48.4605798, 9.044485199999999), (48.4505798, 9.044485199999999, 48.4605798, 9.054485199999998), (48.4505798, 9.054485199999998, 48.4605798, 9.064485199999998), (48.4505798, 9.064485199999998, 48.4605798, 9.074485199999998), (48.4505798, 9.074485199999998, 48.4605798, 9.084485199999998), (48.4505798, 9.084485199999998, 48.4605798, 9.094485199999998), (48.4505798, 9.094485199999998, 48.4605798, 9.104485199999997), (48.4505798, 9.104485199999997, 48.4605798, 9.114485199999997), (48.4505798, 9.114485199999997, 48.4605798, 9.124485199999997), (48.4505798, 9.124485199999997, 48.4605798, 9.1310816000000

## 2. Get places data

In [37]:
# all fields that are possible and interesting for us
fields_extensive = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.displayName", "places.attributions", "places.id", "places.name", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", 
    "places.plusCode", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.shortFormattedAddress", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

# to save quota 
fields_basic = [
    "places.displayName", "places.id", "nextPageToken"
]

In [38]:
def get_places_data(boundary, fields):
    low_lat, low_long, high_lat, high_long = boundary

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": ",".join(fields) 
    }

    results = [] # List to store the results
    page_token = None  # Initialize the page token to None

    while True:
        # JSON request body
        body = {
            "includedType": "restaurant",  # Restrict to restaurants
            "strictTypeFiltering": True,  # Only return results of the specified type
            "textQuery": "restaurant",
            "pageSize": 20,  # max results per page
            "pageToken": page_token,  # Next page token, if any
            "languageCode": "en",  # Language for results
            "locationRestriction": {
                "rectangle": {
                    "low": {"latitude": low_lat, "longitude": low_long},
                    "high": {"latitude": high_lat, "longitude": high_long}
                }}
        }

        # Send the POST request to the API
        response = requests.post(base_url, headers=headers, json=body)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Check if "places" key is present in the response
            if "places" in data:
                results.extend(data["places"])
            else: break # Break the loop if no results are available

            # Check if there is a next page token
            if "nextPageToken" in data:
                page_token = data["nextPageToken"]
            else:
                break
        else:
            print("Error:", response.status_code, "Message:", response.text)
            break
    
    return results

In [35]:
results = get_places_data(tueb_bound, fields_basic)
print(len(results))
print(results[:1])

60
[{'id': 'ChIJS41IiCblmUcRPdcnWt4Nixg', 'displayName': {'text': 'Max-Planck-Haus', 'languageCode': 'de'}}]


In [39]:
results_extensive = get_places_data(tueb_bound, fields_extensive)
print(len(results))
print(results[:1])

60
[{'id': 'ChIJS41IiCblmUcRPdcnWt4Nixg', 'displayName': {'text': 'Max-Planck-Haus', 'languageCode': 'de'}}]


In [58]:
# fields that need further investigation
for result in results_extensive[:5]:
    print(result.get('name', 'No name field'))
    print(result.get('id'))

places/ChIJkcyzhNP6mUcRtktmko5aqu4
ChIJkcyzhNP6mUcRtktmko5aqu4
places/ChIJ8dHXli3lmUcRbSsbGrUNotc
ChIJ8dHXli3lmUcRbSsbGrUNotc
places/ChIJS41IiCblmUcRPdcnWt4Nixg
ChIJS41IiCblmUcRPdcnWt4Nixg
places/ChIJP2j5YRb7mUcRvlrJted0Q_c
ChIJP2j5YRb7mUcRvlrJted0Q_c
places/ChIJ62MugdH6mUcR7IeaAY_mG24
ChIJ62MugdH6mUcR7IeaAY_mG24


In [61]:
for result in results_extensive[:3]:
    print(result.get('addressComponents'))
    print(result.get('formattedAddress'))
    print(result.get('adrFormatAddress'))
    print(result.get('shortFormattedAddress'))
    print(result.get('plusCode'))
    print(result.get('location'))
    print() # empty line

[{'longText': '8', 'shortText': '8', 'types': ['street_number'], 'languageCode': 'en'}, {'longText': 'Lange Gasse', 'shortText': 'Lange G.', 'types': ['route'], 'languageCode': 'de'}, {'longText': 'Tübingen', 'shortText': 'Tübingen', 'types': ['locality', 'political'], 'languageCode': 'de'}, {'longText': 'Tübingen', 'shortText': 'Tübingen', 'types': ['administrative_area_level_3', 'political'], 'languageCode': 'de'}, {'longText': 'Tübingen', 'shortText': 'TÜ', 'types': ['administrative_area_level_2', 'political'], 'languageCode': 'de'}, {'longText': 'Baden-Württemberg', 'shortText': 'BW', 'types': ['administrative_area_level_1', 'political'], 'languageCode': 'de'}, {'longText': 'Germany', 'shortText': 'DE', 'types': ['country', 'political'], 'languageCode': 'en'}, {'longText': '72070', 'shortText': '72070', 'types': ['postal_code'], 'languageCode': 'en'}]
Lange G. 8, 72070 Tübingen, Germany
<span class="street-address">Lange G. 8</span>, <span class="postal-code">72070</span> <span cla

In [71]:
for result in results_extensive[:3]:
    print(result.get('regularOpeningHours'))
    print(result.get('currentOpeningHours'))
    print(result.get('currentSecondaryOpeningHours'))
    print(result.get('regularSecondaryOpeningHours'))
    print() # empty line
    

{'openNow': True, 'periods': [{'open': {'day': 0, 'hour': 10, 'minute': 0}, 'close': {'day': 1, 'hour': 0, 'minute': 0}}, {'open': {'day': 1, 'hour': 10, 'minute': 0}, 'close': {'day': 2, 'hour': 0, 'minute': 0}}, {'open': {'day': 2, 'hour': 10, 'minute': 0}, 'close': {'day': 3, 'hour': 0, 'minute': 0}}, {'open': {'day': 3, 'hour': 10, 'minute': 0}, 'close': {'day': 4, 'hour': 0, 'minute': 0}}, {'open': {'day': 4, 'hour': 10, 'minute': 0}, 'close': {'day': 5, 'hour': 0, 'minute': 0}}, {'open': {'day': 5, 'hour': 10, 'minute': 0}, 'close': {'day': 6, 'hour': 4, 'minute': 0}}, {'open': {'day': 6, 'hour': 10, 'minute': 0}, 'close': {'day': 0, 'hour': 4, 'minute': 0}}], 'weekdayDescriptions': ['Monday: 10:00\u202fAM\u2009–\u200912:00\u202fAM', 'Tuesday: 10:00\u202fAM\u2009–\u200912:00\u202fAM', 'Wednesday: 10:00\u202fAM\u2009–\u200912:00\u202fAM', 'Thursday: 10:00\u202fAM\u2009–\u200912:00\u202fAM', 'Friday: 10:00\u202fAM\u2009–\u20094:00\u202fAM', 'Saturday: 10:00\u202fAM\u2009–\u20094:00

In [66]:
for result in results_extensive[:10]:
    print(result.get('primaryTypeDisplayName'))
    print(result.get('primaryType'))

{'text': 'Restaurant', 'languageCode': 'en'}
restaurant
{'text': 'Restaurant', 'languageCode': 'en'}
restaurant
{'text': 'Restaurant', 'languageCode': 'en'}
restaurant
{'text': 'Turkish restaurant', 'languageCode': 'en'}
turkish_restaurant
{'text': 'Restaurant', 'languageCode': 'en'}
restaurant
{'text': 'Cafe', 'languageCode': 'en'}
cafe
{'text': 'Turkish restaurant', 'languageCode': 'en'}
turkish_restaurant
{'text': 'Restaurant', 'languageCode': 'en'}
restaurant
{'text': 'Restaurant', 'languageCode': 'en'}
restaurant
{'text': 'Restaurant', 'languageCode': 'en'}
restaurant


In [67]:
for result in results_extensive[:5]:
    print(result.get('googleMapsLinks'))

{'directionsUri': "https://www.google.com/maps/dir//''/data=!4m7!4m6!1m1!4e2!1m2!1m1!1s0x4799fad384b3cc91:0xeeaa5a8e92664bb6!3e0", 'placeUri': 'https://maps.google.com/?cid=17197657695455693750', 'writeAReviewUri': 'https://www.google.com/maps/place//data=!4m3!3m2!1s0x4799fad384b3cc91:0xeeaa5a8e92664bb6!12e1', 'reviewsUri': 'https://www.google.com/maps/place//data=!4m4!3m3!1s0x4799fad384b3cc91:0xeeaa5a8e92664bb6!9m1!1b1', 'photosUri': 'https://www.google.com/maps/place//data=!4m3!3m2!1s0x4799fad384b3cc91:0xeeaa5a8e92664bb6!10e5'}
{'directionsUri': "https://www.google.com/maps/dir//''/data=!4m7!4m6!1m1!4e2!1m2!1m1!1s0x4799e52d96d7d1f1:0xd7a20db51a1b2b6d!3e0", 'placeUri': 'https://maps.google.com/?cid=15537996735859862381', 'writeAReviewUri': 'https://www.google.com/maps/place//data=!4m3!3m2!1s0x4799e52d96d7d1f1:0xd7a20db51a1b2b6d!12e1', 'reviewsUri': 'https://www.google.com/maps/place//data=!4m4!3m3!1s0x4799e52d96d7d1f1:0xd7a20db51a1b2b6d!9m1!1b1', 'photosUri': 'https://www.google.com/m

In [68]:
for result in results_extensive[:5]:
    print(result.get('priceRange'))
    print(result.get('priceLevel'))

{'startPrice': {'currencyCode': 'EUR', 'units': '10'}, 'endPrice': {'currencyCode': 'EUR', 'units': '20'}}
PRICE_LEVEL_MODERATE
{'startPrice': {'currencyCode': 'EUR', 'units': '10'}, 'endPrice': {'currencyCode': 'EUR', 'units': '20'}}
PRICE_LEVEL_MODERATE
None
None
{'startPrice': {'currencyCode': 'EUR', 'units': '1'}, 'endPrice': {'currencyCode': 'EUR', 'units': '10'}}
None
{'startPrice': {'currencyCode': 'EUR', 'units': '20'}, 'endPrice': {'currencyCode': 'EUR', 'units': '30'}}
PRICE_LEVEL_MODERATE


In [69]:
for result in results_extensive[:5]:
    print(result.get('containingPlaces'))

None
None
None
[{'name': 'places/ChIJC00Gv8j6mUcRN-FRr8fBLuA', 'id': 'ChIJC00Gv8j6mUcRN-FRr8fBLuA'}]
[{'name': 'places/ChIJUZ1AZQc8mkcRMAalGpHDdPQ', 'id': 'ChIJUZ1AZQc8mkcRMAalGpHDdPQ'}]


drop information: name, addressComponents, shortFormattedAddress, plusCode, currentOpeningHours, regularSecondaryOpeningHours, currentSecondaryOpeningHours

In [ ]:
search_fields = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.displayName", "places.attributions", "places.id", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions",
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

In [ ]:
# to save quota for the further exploration
search_fields = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.displayName", "places.attributions", "places.id", "nextPageToken"]

In [82]:
# final function to get the data
def get_places_data(boundary):
    low_lat, low_long, high_lat, high_long = boundary

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": ",".join(search_fields) 
    }

    results = [] # List to store the results
    page_token = None  # Initialize the page token to None

    while True:
        # JSON request body
        body = {
            "includedType": "restaurant",  # Restrict to restaurants
            "strictTypeFiltering": True,  # Only return results of the specified type
            "textQuery": "restaurant",
            "pageSize": 20,  # max results per page
            "pageToken": page_token,  # Next page token, if any
            "languageCode": "en",  # Language for results
            "locationRestriction": {
                "rectangle": {
                    "low": {"latitude": low_lat, "longitude": low_long},
                    "high": {"latitude": high_lat, "longitude": high_long}
                }}
        }

        # Send the POST request to the API
        response = requests.post(base_url, headers=headers, json=body)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Check if "places" key is present in the response
            if "places" in data:
                results.extend(data["places"])
            else: break # Break the loop if no results are available

            # Check if there is a next page token
            if "nextPageToken" in data:
                page_token = data["nextPageToken"]
            else:
                break
        else:
            print("Error:", response.status_code, "Message:", response.text)
            break
    
    return results

## 3. Get restaurants in grid

In [83]:
def get_places_data_in_grid(grid):
    all_results = []

    for cell in grid:
        results = get_places_data(cell)
        time.sleep(2)

        if len(results) != 60:
            all_results.extend(results)

        # If API returns full results, recursively split the cell
        if len(results) == 60:
            print(f"Overcrowded cell detected: {cell}")
            low_lat, low_long, high_lat, high_long = cell
            mid_lat = (low_lat + high_lat) / 2
            mid_long = (low_long + high_long) / 2
            sub_cells = [
                (low_lat, low_long, mid_lat, mid_long),  # Bottom-left
                (low_lat, mid_long, mid_lat, high_long),  # Bottom-right
                (mid_lat, low_long, high_lat, mid_long),  # Top-left
                (mid_lat, mid_long, high_lat, high_long)  # Top-right
            ]

            # Recursive call for each sub-cell
            subdivided_results = get_places_data_in_grid(sub_cells)
            all_results.extend(subdivided_results)  # Append all subdivided results
    
    return all_results


In [84]:
# test the function when no grid is defined only the boundary of the whole city
all_results = get_places_data_in_grid([tueb_bound])
print(len(all_results))

Overcrowded cell detected: (48.4505798, 8.9644852, 48.5937085, 9.131081600000002)
Overcrowded cell detected: (48.4505798, 9.0477834, 48.52214415, 9.131081600000002)
Overcrowded cell detected: (48.486361975, 9.0477834, 48.52214415, 9.089432500000001)
Overcrowded cell detected: (48.5042530625, 9.0477834, 48.52214415, 9.06860795)
Overcrowded cell detected: (48.51319860625, 9.0477834, 48.52214415, 9.058195675)
135


In [85]:
all_results = get_places_data_in_grid([tueb_bound])
print(len(all_results))

Overcrowded cell detected: (48.4505798, 8.9644852, 48.5937085, 9.131081600000002)
Overcrowded cell detected: (48.4505798, 9.0477834, 48.52214415, 9.131081600000002)
Overcrowded cell detected: (48.486361975, 9.0477834, 48.52214415, 9.089432500000001)
Overcrowded cell detected: (48.5042530625, 9.0477834, 48.52214415, 9.06860795)
Overcrowded cell detected: (48.51319860625, 9.0477834, 48.52214415, 9.058195675)
134


In [86]:
all_results = get_places_data_in_grid([tueb_bound])
print(len(all_results))

Overcrowded cell detected: (48.4505798, 8.9644852, 48.5937085, 9.131081600000002)
Overcrowded cell detected: (48.4505798, 9.0477834, 48.52214415, 9.131081600000002)
Overcrowded cell detected: (48.486361975, 9.0477834, 48.52214415, 9.089432500000001)
Overcrowded cell detected: (48.5042530625, 9.0477834, 48.52214415, 9.06860795)
Overcrowded cell detected: (48.51319860625, 9.0477834, 48.52214415, 9.058195675)
129


We see that eventhough we perform the same search each time the number of restaurants in the output fluctuate. Unfortunatly, this is a known problem of the Google Places API see: https://issuetracker.google.com/issues/119250563?pli=1 

In [88]:
all_results = get_places_data_in_grid(tueb_grid)
print(len(all_results))

200


In [107]:
all_results = get_places_data_in_grid(tueb_grid)
print(len(all_results))

175


In [91]:
tueb_grid2 = divide_area_in_grid(tueb_bound, step_size=0.02)
all_results_grid2 = get_places_data_in_grid(tueb_grid2)
print(len(all_results_grid2))

Overcrowded cell detected: (48.51057980000001, 9.044485199999999, 48.53057980000001, 9.064485199999998)
177


177

In [108]:
b = [48.52057980000001, 9.044485199999999, 48.53057980000001, 9.064485199999998]
res1 = get_places_data_in_grid([b])
print(len(res1))
res2 = get_places_data(b)
print(len(res2))

40
40


### all restaurant in tuebingen

In [ ]:
search_fields = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.displayName", "places.attributions", "places.id", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions",
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

In [111]:
all_restaurants = get_places_data_in_grid(tueb_grid)

## 4. Clean dataset
for a description of each variable go to: https://developers.google.com/maps/documentation/places/web-service/reference/rest/v1/places 

In [120]:
def clean_dataset(results):
    cleaned_basics = []
    cleaned_general_info = []
    cleaned_restaurant_specific = []
    cleaned_user_specific = []

    for result in results:
        # the most basic information
        basic_info = {
            "id": result.get("id"),
            "name": result.get("displayName")}
        cleaned_basics.append(basic_info)

        # general information
        general_info = {
            "id": result.get("id"),
            "name": result.get("displayName"),
            "attributions": result.get("attributions"),
            "types": result.get("types"),
            "primaryType": result.get("primaryType"),
            "primaryTypeDisplayName": result.get("primaryTypeDisplayName"),
            "businessStatus": result.get("businessStatus"),
            "containingPlaces": result.get("containingPlaces"),
            "pureServiceAreaBusiness": result.get("pureServiceAreaBusiness"),
            "formattedAddress": result.get("formattedAddress"),
            "location": result.get("location"),
            "subDestinations": result.get("subDestinations"),
            "nationalPhoneNumber": result.get("nationalPhoneNumber"),
            "internationalPhoneNumber": result.get("internationalPhoneNumber"),
            "placesWebsiteUri": result.get("websiteUri"),
            "editorialSummary": result.get("editorialSummary"),
            "regularOpeningHours": result.get("regularOpeningHours"),
            "utcOffsetMinutes": result.get("utcOffsetMinutes"),
            "priceLevel": result.get("priceLevel"),
            "priceRange": result.get("priceRange"),
            "rating": result.get("rating"),
            "userRatingCount": result.get("userRatingCount"),
            "reviews": result.get("reviews")}
        cleaned_general_info.append(general_info)

        # restaurant specific information
        restaurant_info = {
            "id": result.get("id"),
            "name": result.get("displayName"),
            "curbsidePickup": result.get("curbsidePickup"),
            "delivery": result.get("delivery"),
            "dineIn": result.get("dineIn"),
            "evChargeOptions": result.get("evChargeOptions"),
            "fuelOptions": result.get("fuelOptions"),
            "liveMusic": result.get("liveMusic"),
            "parkingOptions": result.get("parkingOptions"),
            "paymentOptions": result.get("paymentOptions"),
            "outdoorSeating": result.get("outdoorSeating"),
            "reservable": result.get("reservable"),
            "restroom": result.get("restroom"),
            "servesBeer": result.get("servesBeer"),
            "servesBreakfast": result.get("servesBreakfast"),
            "servesBrunch": result.get("servesBrunch"),
            "servesCocktails": result.get("servesCocktails"),
            "servesCoffee": result.get("servesCoffee"),
            "servesDessert": result.get("servesDessert"),
            "servesDinner": result.get("servesDinner"),
            "servesLunch": result.get("servesLunch"),
            "servesVegetarianFood": result.get("servesVegetarianFood"),
            "servesWine": result.get("servesWine"),
            "takeout": result.get("takeout")}
        cleaned_restaurant_specific.append(restaurant_info)

        # user specific information
        user_info = {
            "id": result.get("id"),
            "name": result.get("displayName"),
            "accessibilityOptions": result.get("accessibilityOptions"),
            "allowsDogs": result.get("allowsDogs"),
            "goodForChildren": result.get("goodForChildren"),
            "goodForGroups": result.get("goodForGroups"),
            "goodForWatchingSports": result.get("goodForWatchingSports"),
            "menuForChildren": result.get("menuForChildren")}
        cleaned_user_specific.append(user_info)

    cleaned_basics = pd.DataFrame(cleaned_basics)
    cleaned_general_info = pd.DataFrame(cleaned_general_info)
    cleaned_restaurant_specific = pd.DataFrame(cleaned_restaurant_specific)
    cleaned_user_specific = pd.DataFrame(cleaned_user_specific)
    return cleaned_basics, cleaned_general_info, cleaned_restaurant_specific, cleaned_user_specific


In [122]:
TUE_basics, TUE_general, TUE_restaurant_specific, TUE_user_specific = clean_dataset(all_restaurants)

# save the data in a csv file
TUE_basics.to_csv("TUE_basics.csv", index=False)
TUE_general.to_csv("TUE_general.csv", index=False)
TUE_restaurant_specific.to_csv("TUE_restaurant_specific.csv", index=False)
TUE_user_specific.to_csv("TUE_user_specific.csv", index=False)